# Chapter 7: Properties, Indexers, and Records

In Chapter 6, you learned how to define classes with fields and methods, and you saw the importance of encapsulation. Now we’ll dive deeper into three C# features that make encapsulation more powerful and expressive: **properties**, **indexers**, and **records**.

Properties allow you to expose data with control, hiding implementation details while providing a field‑like syntax. Indexers let your objects behave like arrays, enabling you to access elements by index or key. Records, introduced in C# 9, are a special kind of class designed for immutable data and value‑based equality.

By the end of this chapter, you’ll be able to design classes that are both safe and intuitive to use.

---

## 7.1 Properties Revisited – Auto‑Implemented and Full Properties

As you saw in Chapter 6, properties are members that provide a flexible way to read, write, or compute the values of private fields. They consist of `get` and `set` accessors.

### Auto‑Implemented Properties

The simplest form is the **auto‑implemented property**, where the compiler generates a hidden backing field for you.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
}
```

This is concise and perfect when you don’t need validation or additional logic. The backing field is not accessible directly; you always go through the property.

### Full Property Implementation

If you need validation, change notification, or computed behavior, you can provide a full implementation with an explicit backing field.

```csharp
private string _name;
public string Name
{
    get { return _name; }
    set
    {
        if (string.IsNullOrWhiteSpace(value))
            throw new ArgumentException("Name cannot be empty");
        _name = value;
    }
}
```

### Computed Properties (Read‑Only)

A property can have only a `get` accessor, making it read‑only. Its value is typically computed from other fields.

```csharp
public class Rectangle
{
    public double Width { get; set; }
    public double Height { get; set; }
    public double Area => Width * Height; // expression‑bodied property
}
```

Here, `Area` is computed on the fly; there is no backing field. Such a property is often called a **computed property** or **derived property**.

### Property Access Modifiers

You can apply access modifiers to the `get` or `set` accessors individually, allowing finer control. For example, you might want a public getter but a private setter.

```csharp
public class Person
{
    public string Name { get; private set; } // can be set only inside the class

    public Person(string name)
    {
        Name = name; // allowed because we're inside the class
    }
}
```

The `init` accessor (C# 9) is a special setter that can only be called during object initialization – either in a constructor or an object initializer. After that, the property becomes read‑only.

```csharp
public class Person
{
    public string Name { get; init; }
}

var p = new Person { Name = "Alice" }; // OK
p.Name = "Bob"; // Compiler error – init‑only property cannot be set after initialization
```

This is perfect for creating immutable objects without writing a full constructor for every property.

---

## 7.2 Indexers

An **indexer** allows an object to be indexed like an array. You define an indexer using the `this` keyword followed by a parameter list in square brackets.

### Basic Indexer Example

Imagine a class that represents a collection of temperatures:

```csharp
public class TempRecord
{
    private float[] temps = new float[10] { 56.2F, 56.7F, 56.5F, 57.1F, 57.3F, 57.6F, 57.8F, 58.0F, 58.2F, 58.5F };

    public float this[int index]
    {
        get => temps[index];
        set => temps[index] = value;
    }
}
```

You can now use it like an array:

```csharp
TempRecord record = new TempRecord();
record[3] = 58.1F;
Console.WriteLine(record[3]);
```

### Indexers with Different Parameter Types

Indexers aren’t limited to integers. You can use any type, such as a string, to create a dictionary‑like behavior.

```csharp
public class DayTemperatures
{
    private Dictionary<string, float> temps = new Dictionary<string, float>();

    public float this[string day]
    {
        get => temps.ContainsKey(day) ? temps[day] : throw new KeyNotFoundException();
        set => temps[day] = value;
    }
}

// Usage
DayTemperatures week = new DayTemperatures();
week["Monday"] = 72.5F;
week["Tuesday"] = 74.0F;
Console.WriteLine(week["Monday"]);
```

### Overloading Indexers

A class can have multiple indexers as long as their parameter lists differ.

```csharp
public class MultiIndex
{
    public string this[int i] => "int index";
    public string this[string s] => "string index";
}
```

### Indexers in Real‑World Classes

Many .NET collections use indexers: `List<T>` has an integer indexer, `Dictionary<TKey, TValue>` has a key‑based indexer. When you create your own collection‑like classes, indexers make them intuitive to use.

---

## 7.3 Records – Immutable Data Objects

**Records** are a reference type (like a class) introduced in C# 9 that provides built‑in functionality for immutable data and value‑based equality. They are ideal for DTOs (Data Transfer Objects), API responses, and any situation where you want objects that are primarily about data and shouldn’t change after creation.

### Declaring a Record

The simplest way to declare a record is with the `record` keyword and a positional syntax:

```csharp
public record Person(string FirstName, string LastName);
```

This single line creates a record with:

- Two public **init‑only properties** `FirstName` and `LastName`.
- A primary constructor that sets those properties.
- Deconstruct method for pattern matching.
- Value‑based equality members (`Equals`, `GetHashCode`, `==`, `!=`).
- A nice `ToString()` implementation.

You can use it like this:

```csharp
var person1 = new Person("Alice", "Smith");
var person2 = new Person("Alice", "Smith");

Console.WriteLine(person1); // Person { FirstName = Alice, LastName = Smith }
Console.WriteLine(person1 == person2); // True (value equality)
```

### Traditional Record Syntax

You can also write a record with a more traditional class‑like syntax, giving you full control:

```csharp
public record Person
{
    public string FirstName { get; init; }
    public string LastName { get; init; }

    public Person(string firstName, string lastName)
    {
        FirstName = firstName;
        LastName = lastName;
    }
}
```

But the positional syntax is much more concise for simple data containers.

### The `with` Expression

Records support non‑destructive mutation via the `with` expression, which creates a new record instance with some properties changed.

```csharp
var original = new Person("Alice", "Smith");
var modified = original with { LastName = "Jones" };

Console.WriteLine(original); // Person { FirstName = Alice, LastName = Smith }
Console.WriteLine(modified); // Person { FirstName = Alice, LastName = Jones }
```

The original remains unchanged – perfect for immutability.

### Value‑Based Equality

Unlike classes, which use reference equality by default, records use **value equality**: two records are equal if all their properties are equal. This is implemented by the compiler overriding `Equals`, `GetHashCode`, and the `==` and `!=` operators.

```csharp
var a = new Person("Alice", "Smith");
var b = new Person("Alice", "Smith");
Console.WriteLine(a == b); // True
Console.WriteLine(ReferenceEquals(a, b)); // False (they are different objects)
```

### When to Use Records vs. Classes

- Use **records** when your object is primarily a container for data that should be immutable and compared by value (e.g., configuration, DTOs, events).
- Use **classes** when your object has identity, mutable state, or complex behavior (e.g., entities in a domain model, services).

Records can also be mutable if you use `set` instead of `init`, but that defeats their purpose. Stick to immutability for records.

### Records Can Have Methods

Records aren’t just data bags; they can contain methods, properties, and even other members. For example:

```csharp
public record Person(string FirstName, string LastName)
{
    public string FullName => $"{FirstName} {LastName}";
}
```

### Inheritance with Records

Records can inherit from other records, but not from classes (and vice versa).

```csharp
public record Employee(string FirstName, string LastName, string Department) 
    : Person(FirstName, LastName);
```

Equality and `with` expressions work correctly with inheritance.

---

## 7.4 Putting It All Together: A Practical Example

Let’s build a simple library system that demonstrates properties (including computed and init‑only), an indexer, and a record.

```csharp
using System;
using System.Collections.Generic;

namespace LibrarySystem
{
    // Record for a book (immutable data)
    public record Book
    {
        public string Title { get; init; }
        public string Author { get; init; }
        public string ISBN { get; init; }
        public int YearPublished { get; init; }

        public Book(string title, string author, string isbn, int year)
        {
            if (string.IsNullOrWhiteSpace(title)) throw new ArgumentException("Title required");
            if (string.IsNullOrWhiteSpace(author)) throw new ArgumentException("Author required");
            if (string.IsNullOrWhiteSpace(isbn)) throw new ArgumentException("ISBN required");
            if (year < 0 || year > DateTime.Now.Year + 1) throw new ArgumentException("Invalid year");

            Title = title;
            Author = author;
            ISBN = isbn;
            YearPublished = year;
        }

        // Computed property
        public string DisplayName => $"{Title} by {Author} ({YearPublished})";
    }

    // Class representing a library collection
    public class Library
    {
        private List<Book> _books = new List<Book>();

        // Read‑only property exposing the collection safely
        public IReadOnlyList<Book> Books => _books.AsReadOnly();

        // Indexer to access a book by ISBN (string key)
        public Book this[string isbn]
        {
            get
            {
                var book = _books.Find(b => b.ISBN == isbn);
                if (book == null)
                    throw new KeyNotFoundException($"Book with ISBN {isbn} not found");
                return book;
            }
        }

        // Indexer to access a book by index (position)
        public Book this[int index]
        {
            get
            {
                if (index < 0 || index >= _books.Count)
                    throw new IndexOutOfRangeException();
                return _books[index];
            }
        }

        public void AddBook(Book book)
        {
            if (book == null) throw new ArgumentNullException(nameof(book));
            // Ensure no duplicate ISBN
            if (_books.Exists(b => b.ISBN == book.ISBN))
                throw new InvalidOperationException("Book with same ISBN already exists");
            _books.Add(book);
        }

        public bool RemoveBook(string isbn)
        {
            var book = _books.Find(b => b.ISBN == isbn);
            if (book != null)
                return _books.Remove(book);
            return false;
        }

        // Find books by author – returns array of matching books
        public Book[] FindByAuthor(string author)
        {
            return _books.FindAll(b => b.Author.Equals(author, StringComparison.OrdinalIgnoreCase)).ToArray();
        }
    }

    class Program
    {
        static void Main()
        {
            Library library = new Library();

            // Add some books (using object initializer with init‑only properties)
            library.AddBook(new Book("1984", "George Orwell", "978-0451524935", 1949));
            library.AddBook(new Book("Brave New World", "Aldous Huxley", "978-0060850524", 1932));
            library.AddBook(new Book("Fahrenheit 451", "Ray Bradbury", "978-1451673319", 1953));

            // Use indexers
            Console.WriteLine("Book at index 1: " + library[1].DisplayName);
            Console.WriteLine("Book with ISBN 978-0060850524: " + library["978-0060850524"].DisplayName);

            // Find by author
            Console.WriteLine("\nBooks by Aldous Huxley:");
            foreach (var book in library.FindByAuthor("Aldous Huxley"))
            {
                Console.WriteLine($"  {book.DisplayName}");
            }

            // Demonstrate with expression on a record (non‑destructive mutation)
            var originalBook = library[0];
            var updatedBook = originalBook with { YearPublished = 1950 }; // create a modified copy
            Console.WriteLine($"\nOriginal: {originalBook.DisplayName}");
            Console.WriteLine($"Modified: {updatedBook.DisplayName}");

            // Show equality (two different objects with same data)
            var book1 = new Book("1984", "George Orwell", "978-0451524935", 1949);
            var book2 = new Book("1984", "George Orwell", "978-0451524935", 1949);
            Console.WriteLine($"book1 == book2? {book1 == book2}"); // True (value equality)
        }
    }
}
```

**Explanation:**

- `Book` is a **record** with init‑only properties, validation in the constructor, and a computed property `DisplayName`.
- `Library` is a class that contains a private list of books. It exposes a read‑only list via a property, ensuring external code cannot modify the collection directly (though the books themselves are immutable records).
- Two **indexers** are defined: one by ISBN (string) and one by position (int). This makes the library intuitive to use.
- Methods `AddBook`, `RemoveBook`, and `FindByAuthor` provide controlled manipulation.
- The `Main` method demonstrates all features, including the `with` expression to create a modified copy of a record, and value‑based equality of records.

---

## 7.5 Common Pitfalls and Best Practices

### 1. Prefer Auto‑Properties for Simple Data

If a property has no logic, use auto‑implemented properties. They are concise and expressive.

### 2. Use Computed Properties Instead of Methods for Simple Calculations

If a value can be derived from existing data and the calculation is lightweight, a computed property is more natural than a method (e.g., `Area` vs. `CalculateArea()`).

### 3. Make Properties Read‑Only When Appropriate

If a property should not change after object creation, use `get; init;` (for records) or `get; private set;` (for classes) to enforce immutability.

### 4. Use Indexers for Collection‑Like Classes

If your class represents a collection of items, provide an indexer to make it feel like a native array or list.

### 5. Keep Indexers Simple

Indexers should be straightforward and not perform heavy computation. If an indexer might throw, document it clearly.

### 6. Favor Records for Immutable Data

Records reduce boilerplate and give you value equality for free. Use them whenever you need simple data containers.

### 7. Be Aware of Record Equality Semantics

Record equality compares all properties. If you have properties that shouldn’t be part of equality (e.g., a cached value), you might need to override equality members manually (rare).

### 8. Use `with` Expressions Safely

`with` creates a shallow copy. If your record contains reference types (other than strings), the copy shares references to the same objects. For deep immutability, consider using records for nested data as well.

### 9. Validate Data in Constructors

Even with init‑only properties, you can (and should) validate in the constructor to ensure objects are always in a valid state.

### 10. Name Indexers Clearly

Although indexers use `this`, the class name itself should convey what the indexer does. For example, a `Library` class with an indexer by ISBN is clear.

---

## 7.6 Chapter Summary

In this chapter, you expanded your knowledge of C# types with three powerful features:

- **Properties** – you revisited auto‑implemented and full properties, learned about computed properties, and saw how to control access with `init` and private setters.
- **Indexers** – you discovered how to make your classes indexable, with parameters of any type, and how they improve usability for collection‑like classes.
- **Records** – you explored the modern way to define immutable data objects, with built‑in value equality and non‑destructive mutation via `with` expressions.

Together, these tools enable you to design classes that are both robust and pleasant to use. They are widely used in modern C# development, especially in scenarios involving data transfer, configuration, and API design.

In the next chapter, **Constructors & Object Initialization**, we’ll dive deep into how objects come to life. You’ll learn about different types of constructors, constructor chaining, object initializers, and the `required` modifier (C# 11). These concepts will give you fine‑grained control over how objects are created and initialized.

**Exercises:**

1. Create a `Student` record with properties `FirstName`, `LastName`, `StudentId`, and `GPA`. Ensure `GPA` is between 0.0 and 4.0.
2. Define a `Classroom` class that contains a list of students. Add an indexer that allows access by student ID (string). Also add a method to get the average GPA.
3. Create a `Matrix` class that uses a 2D array internally and provides an indexer with two integers (row, column) to get and set values.
4. Experiment with the `with` expression: create a record, modify a copy, and verify the original is unchanged.
5. Write a program that demonstrates value equality with records vs. reference equality with classes by comparing two objects with identical data.

Now, let’s move on to Chapter 8, where we’ll explore the fascinating world of constructors and object initialization!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. classes_and_objects.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. constructors_object_initialization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
